In [23]:
%matplotlib widget

import pandas
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt

calibration_data = pandas.read_feather('data_dump.bin')
test_data = pandas.read_feather('data_dump_2.bin')

In [24]:
calibration_data.describe()

,x,y,z
count,206417.000000,206417.000000,206417.000000
mean,0.081842,-0.028652,-0.890410
std,0.028038,0.025647,0.031082
min,-0.384000,-0.512000,-1.408000
25%,0.064000,-0.048000,-0.912000
50%,0.080000,-0.032000,-0.888000
75%,0.096000,0.000000,-0.872000
max,0.232000,0.088000,-0.744000


In [25]:
test_data.describe()

,x,y,z
count,43155.000000,43155.000000,43155.000000
mean,0.076652,-0.024042,-0.894116
std,0.106759,0.049641,0.042082
min,-0.832000,-0.464000,-1.160000
25%,0.048000,-0.048000,-0.920000
50%,0.080000,-0.024000,-0.896000
75%,0.112000,0.000000,-0.872000
max,0.864000,0.440000,-0.600000


In [26]:
plt.plot(test_data['x'], test_data['y'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
rot_x_dps = np.arctan2(calibration_data['x'], np.hypot(calibration_data['y'], calibration_data['z']))
avg_rot_x = np.average(rot_x_dps)

rot_y_dps = np.arctan2(calibration_data['y'], np.hypot(calibration_data['x'], calibration_data['z']))
avg_rot_y = np.average(rot_y_dps)


pitch = avg_rot_y
roll = avg_rot_x

print(pitch)
print(roll)

-0.03196562225229186
0.09153202247604791


In [28]:
def get_rotation_matrix(yaw, pitch, roll):
    return np.array([
        [np.cos(roll) * np.cos(pitch), np.cos(pitch) * np.sin(roll), -np.sin(pitch)],
        [np.cos(roll) * np.sin(yaw) * np.sin(pitch) - np.cos(yaw) * np.sin(roll),
         np.cos(yaw) * np.cos(roll) + np.sin(yaw) * np.sin(roll) * np.sin(pitch), np.cos(pitch) * np.sin(yaw)],
        [np.sin(yaw) * np.sin(roll) + np.cos(yaw) * np.cos(roll) * np.sin(pitch),
         np.cos(yaw) * np.sin(roll) * np.sin(pitch) - np.cos(roll) * np.sin(yaw), np.cos(yaw) * np.cos(pitch)]])


# Attempt to remove rotation from the dataframe
rot_removed_df = calibration_data @ get_rotation_matrix(0, -pitch, -roll)
rot_removed_df.columns = ['x', 'y', 'z']

print(rot_removed_df)

               x         y         z
0       0.096976 -0.041036 -0.867650
1       0.093542 -0.072855 -0.883641
2       0.040221 -0.035826 -0.897843
3       0.080065 -0.047517 -0.875134
4       0.049580 -0.004551 -0.946075
...          ...       ...       ...
206412  0.007607 -0.032833 -0.920808
206413  0.054905 -0.045208 -0.914347
206414  0.038758 -0.051759 -0.897843
206415  0.064872 -0.038089 -0.874623
206416  0.067731  0.009850 -0.922598

[206417 rows x 3 columns]


In [29]:
# Remove gravity from calibration matrix and compute gains
calibration_gains = np.average(rot_removed_df + np.array([0, 0, 1]), axis=0)
calibration_std = np.array(np.std(rot_removed_df))
print(calibration_gains)
print(f"Calibration STD: {calibration_std}")

[ 0.05050059 -0.03340827  0.10742891]
Calibration STD: [0.02708322 0.02660997 0.0311185 ]


In [30]:
# Stupid me did not record time - so guess a dt // that will only really scale it
dt = 1/500

v_dps = []
p_dps = []

v = np.zeros(3, dtype=np.float64)
p = np.zeros(3, dtype=np.float64)

# Perform semi-implicit euler integration using 2nd derivative/acceleration
for i in range(len(test_data)):
    measurement = test_data.loc[i] - calibration_gains
    #print(measurement)
    v += measurement * dt
    p += v * dt
    
    v_dps.append(v.copy())
    p_dps.append(p.copy())

# Turn integrated points into NumPy arrays
v_np = np.array(v_dps)
p_np = np.array(p_dps)

#print(p_np)
#print(p_np[:, 0], p_np[:, 1])

pos_df = pandas.DataFrame(p_np)

plt.scatter(p_np[:, 0], p_np[:, 1])

pos_df.describe()

,0,1,2
count,4.315500e+04,4.315500e+04,43155.000000
mean,3.238477e+01,1.272017e+01,-1243.815674
std,2.942399e+01,1.108419e+01,1112.266917
min,2.779976e-07,1.336331e-07,-3730.576606
25%,5.569911e+00,2.582301e+00,-2098.794778
50%,2.388727e+01,9.881382e+00,-933.121702
75%,5.488224e+01,2.140437e+01,-233.414004
max,9.864071e+01,3.697312e+01,-0.000004


In [31]:
%matplotlib widget

# Plot a vector field showing velocity in all points
plt.quiver(p_np[:, 0], p_np[:, 1], v_np[:, 0], v_np[:, 1], scale=1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
# First measurement data processing - gets rid of gravity, rotation and applies calibration gains
p1_measurement_data = test_data @ get_rotation_matrix(0, -pitch, -roll) - calibration_gains + [0, 0, 1]
p1_measurement_data.columns = ['x', 'y', 'z']

p1_measurement_data.describe()

,x,y,z
count,43155.000000,43155.000000,43155.000000
mean,-0.004862,0.005077,-0.003538
std,0.106116,0.051205,0.041834
min,-0.903677,-0.468141,-0.255586
25%,-0.031497,-0.019035,-0.030027
50%,-0.002572,0.004841,-0.004761
75%,0.026426,0.030062,0.021017
max,0.783473,0.483737,0.284440


In [33]:
# Filter out DPs that are less than 2 stdevs from a 0 value. Also include DP number 

NUM_STDEVS = 0
p2_measurement_data = p1_measurement_data[np.logical_and(np.abs(p1_measurement_data['x']) > NUM_STDEVS*calibration_std[0], np.abs(p1_measurement_data['y']) > NUM_STDEVS*calibration_std[1])]
p2_measurement_data.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [34]:
# Filter out datapoints that 
filtered_data = p2_measurement_data
filtered_data

,x,y,z
0,0.039910,0.025110,-0.022798
1,-0.011488,-0.018374,0.002724
2,0.028292,-0.013992,-0.022543
3,-0.021752,0.030770,0.043215
4,0.009399,0.011843,0.066180
...,...,...,...
43150,-0.475373,-0.032031,0.113504
43151,-0.293597,-0.072816,-0.116518
43152,-0.353608,-0.035174,-0.090485
43153,-0.102666,-0.066241,0.053512


In [35]:
v_dps = []
p_dps = []

v = np.zeros(3, dtype=np.float64)
p = np.zeros(3, dtype=np.float64)

# Perform semi-implicit euler integration using 2nd derivative/acceleration
last_i = 0

DT_PER_I = 1/100

for i in filtered_data.index:
    measurement = filtered_data.loc[i]
    
    # Compute dt
    dt = (i - last_i) * DT_PER_I
    last_i = i
    
    #print(measurement)
    #v += (measurement - -0.006280741567885446) * dt
    v += measurement * dt
    p += v * dt
    
    v_dps.append(v.copy())
    p_dps.append(p.copy())

# Turn integrated points into NumPy arrays
v_np = np.array(v_dps)
p_np = np.array(p_dps)

p_np

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.14878595e-06, -1.83744322e-06,  2.72359789e-07],
       [ 5.31602837e-07, -5.07409773e-06, -1.70953529e-06],
       ...,
       [-4.17522571e+02,  5.21650244e+02, -3.31003507e+02],
       [-4.17543547e+02,  5.21672148e+02, -3.31018776e+02],
       [-4.17564535e+02,  5.21694054e+02, -3.31034041e+02]])

In [36]:
%matplotlib widget

pos_df = pandas.DataFrame(p_np)
plt.scatter(p_np[:, 0], p_np[:, 1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
%matplotlib widget

plt.scatter(v_np[:, 0], v_np[:, 1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
%matplotlib widget
filtered_data_np = np.array(filtered_data)

axis = 0

plt.subplot(3, 1, 1)
plt.title("Acceleration")
plt.plot(filtered_data_np[:, axis])

plt.subplot(3, 1, 2)
plt.title("Velocity")
plt.plot(v_np[:, axis])

plt.subplot(3, 1, 3)
plt.title("Position")
plt.plot(p_np[:, axis])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
np.average(filtered_data_np[:, 0])

-0.004862369590036541

In [40]:
# NOTE: I am aware accel data is still in g's, but that only really scales things

In [53]:
%matplotlib widget

# Just work with x datapoints to make life easier (assume 100Hz const sample rate )
t_dps = np.array(list(range(len(filtered_data_np)))) / 100
ax_dps = filtered_data_np[:, 0]

# Get the samples that almost certainly lie within the noise (-> deviate less than 3.5 STD)
noise_thres = 3.5 * calibration_std[0]
noise_thres_dps = ax_dps[np.abs(ax_dps) < noise_thres]
noise_avg = np.average(noise_thres_dps)

plt.plot(t_dps, ax_dps, zorder=1)
plt.axhline(noise_avg, np.min(t_dps), np.max(t_dps), color='orange')
plt.fill_between(t_dps, -noise_thres, +noise_thres, color='orange', alpha=0.5, zorder=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
%matplotlib widget

# After each acceleration, it doesn't quite return to 0, but stays below consistently -> add a fudgefactor to all measurements pre-filter to see if this changes anything
FILTER_BEFORE_FUDGE_ADD = 0.0030956915839844986 + 0.0021994351769940397 + 0.0015585331691145782 + 0.0010966370369129227

# Get all of the DPs excludinng the noise below the above threshold
ax_dps_with_before_fudge = ax_dps.copy() + FILTER_BEFORE_FUDGE_ADD
above_noise_thres_indices = np.where(np.abs(ax_dps_with_before_fudge) > noise_thres)
above_noise_thres_dps = ax_dps_with_before_fudge[above_noise_thres_indices]

# Zero the DPs suspected of being noise
zeroed_noise_dps = ax_dps_with_before_fudge.copy()
zeroed_noise_dps[np.abs(ax_dps_with_before_fudge) < noise_thres] = 0

plt.plot(above_noise_thres_indices[0], above_noise_thres_dps)
plt.plot(list(range(len(zeroed_noise_dps))), zeroed_noise_dps)

print(np.average(above_noise_thres_dps))
print(np.average(zeroed_noise_dps))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.003996257703299634
-0.0007672111011896066


In [73]:
%matplotlib widget
# Helper function for simulating and plotting 1D kinematics
def perform_and_plot_1D_sim(time_dps, acceleration_dps):
    # Ensure time and acceleration datapoints have the same dimensions
    assert(len(time_dps) == len(acceleration_dps))
    
    # Set up initial conditions and run iterative sim
    print("Running simulation...")
    v = 0
    p = 0
    v_dps = []
    p_dps = []
    last_t = time_dps[0]  # NOTE: This means the first DP won't do much as dt will be 0, but considering we don't have to start at t=0, it's probably still the best solution
    for i in range(len(time_dps)):
        cur_t = time_dps[i]
        dt = cur_t - last_t
        last_t = cur_t
        
        v += acceleration_dps[i] * dt
        p += v * dt
        
        v_dps.append(v.copy())
        p_dps.append(p.copy())
        
        if (i % 1000 == 0):
            print(i)
        
    print("Plotting...")
    plt.subplot(3, 1, 1)
    plt.title("Acceleration")
    plt.plot(time_dps, acceleration_dps)

    plt.subplot(3, 1, 2)
    plt.title("Velocity")
    plt.plot(time_dps, v_dps)

    plt.subplot(3, 1, 3)
    plt.title("Position")
    plt.plot(time_dps, p_dps)
    
    print("done!")


perform_and_plot_1D_sim(np.array(list(range(len(p_dps))), dtype=np.float64) / 3000, zeroed_noise_dps * 9.81)

Running simulation...
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
Plotting...


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

done!


In [ ]:
%matplotlib widget

# Based on: https://realpython.com/python-scipy-fft/
from scipy.fft import fft, fftfreq, irfft
SAMPLE_RATE = 100
DURATION = len(t_dps) / SAMPLE_RATE
N = int(SAMPLE_RATE * DURATION)
yf = fft(ax_dps)
xf = fftfreq(N, 1 / SAMPLE_RATE)

plt.plot(xf, np.abs(yf))

In [ ]:
# Similar to last one, but now on the calibration data set's x dps
%matplotlib widget

SAMPLE_RATE = 3600
DURATION = len(rot_removed_df) / SAMPLE_RATE
N = int(SAMPLE_RATE * DURATION)
yf = fft(np.array(rot_removed_df)[:, 0])
xf = fftfreq(N, 1 / SAMPLE_RATE)

plt.plot(xf, np.abs(yf))

In [ ]:
%matplotlib widget
from scipy import signal
sos_filter = signal.butter(20, 0.00001, 'lp', fs=1/SAMPLE_RATE, output='sos')
filtered = signal.sosfilt(sos_filter, ax_dps)


plt.plot(t_dps, filtered, zorder=1)

In [ ]:
# NOTE: Because we end stationary/don't rotate, the sum of all acceleration (times the timestep, which is assumed constant -> can drop out) should be 0 -> we can just average the entire dataset under these idealized assumptions
filtered_normed = filtered - np.average(filtered)

In [ ]:
np.std(filtered_normed)

In [ ]:
# Run another FFT on the filtered data to see what kind of difference it made
%matplotlib widget

# Based on: https://realpython.com/python-scipy-fft/
from scipy.fft import fft, fftfreq
SAMPLE_RATE = 100
DURATION = len(filtered_normed) / SAMPLE_RATE
N = int(SAMPLE_RATE * DURATION)
yf = fft(filtered_normed)
xf = fftfreq(N, 1 / SAMPLE_RATE)

plt.plot(xf, np.abs(yf))

In [ ]:
%matplotlib widget
dt = 1/SAMPLE_RATE
x = 0
vx = 0

t_dps = []
x_dps = []
vx_dps = []

for i in range(len(filtered_normed)):
    vx += filtered_normed[i] * dt
    x += vx * dt
    
    t_dps.append(i * dt)
    x_dps.append(x)
    vx_dps.append(vx)

plt.plot(t_dps, x_dps)


In [ ]:
%matplotlib widget
# NOTE: FFT appears to change magnitude as well, or at least, it appears to do so when filtering frequencies
from scipy.fft import ifft
signal_thres = 13
new_filtered_signal = ifft(yf[np.abs(yf) > signal_thres])
plt.plot(new_filtered_signal.real)

In [ ]:
f2_dps = new_filtered_signal.real

In [ ]:
%matplotlib widget
f3_dps = signal.sosfilt(sos_filter, f2_dps)
plt.plot(f3_dps)

In [ ]:
%matplotlib widget
plt.plot(f2_dps)

In [ ]:
np.average(f2_dps)

In [ ]:
%matplotlib widget
plt.plot(t_dps, ax_dps)

In [ ]:
%matplotlib widget
import scipy.signal
savgol_filtered = signal.savgol_filter(ax_dps, 51, 10, deriv=0, delta=1.0, axis=- 1, mode='interp', cval=0.0)


plt.plot(t_dps, ax_dps)
plt.plot(t_dps, savgol_filtered)